In [162]:
#初期設定
import os
import pandas as pd
import numpy as np
import time
import lxml
import re
import requests
from bs4 import BeautifulSoup as bs4
import tqdm
import sys
import inspect
# chromeのwebdriver自動更新用
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


In [ ]:
pd.set_option('display.max_columns', 20)

In [163]:
# https://web-kiwami.com/python-beautyfulsoup4.html
# http://kondou.com/BS4/
# bs4参考

In [181]:
# 1
# all_year
this_year = 2022 #年変わったら変更
get_year = list(range(2011,this_year+1))
df_scraping_companies = pd.DataFrame()
dict1 = {}
for year in get_year:
    if 2011 <= year <= 2017:
        print(year, "出力中")
        # 単年のデータ取得2011~2017
        url =  rf"https://www.ipokiso.com/company/{year}.html"
        html_res = requests.get(url)
        # ページアクセスエラーの出力
        if html_res.status_code != 200:
            print("requests.getでのurlのアクセスができていません")
            lineno = inspect.currentframe().f_lineno
            print(f"エラーが発生しました。行番号: {lineno}")
            sys.exit()
        soup = bs4(html_res.content, 'html.parser')
        find_all_list = soup.find_all(href=re.compile("company/[0-9]{4}/"))
        url_dict = {
            find_all_list[i].text:"https://www.ipokiso.com/" + find_all_list[i]["href"]
            for i in range(len(find_all_list))
        }
        dict1.update(url_dict)
        dfs_list = pd.read_html(url)
        time.sleep(np.random.randint(100,120)/100)
        for i in range(len(dfs_list)):
            dfs_list[i]["上場年"] = f"{year}"
            df_scraping_companies = pd.concat([df_scraping_companies, dfs_list[i]])
            df_scraping_companies = df_scraping_companies.reset_index(drop=True)
        # df_scraping_companies.drop(df_scraping_companies[df_scraping_companies['企業名'] == "企業名"].index , inplace=True)
        # df_scraping_companies.drop(df_scraping_companies[df_scraping_companies['初値'] == "初値"].index , inplace=True)
    else:
        # 単年のデータ取得2018~2022
        print(year, "出力中")
        url = rf"https://www.ipokiso.com/company/{year}.html"
        if year == 2022:
            url = r"https://www.ipokiso.com/company/index.html"
        html_res = requests.get(url)
        # ページアクセスエラーの出力
        if html_res.status_code != 200:
            print("requests.getでのurlのアクセスができていません")
            lineno = inspect.currentframe().f_lineno
            print(f"エラーが発生しました。行番号: {lineno}")
            sys.exit()
        soup = bs4(html_res.content, 'html.parser')
        find_all_list = soup.find_all(href=re.compile("company/[0-9]{4}/"))
        url_dict = {
            find_all_list[i].text:"https://www.ipokiso.com/" + find_all_list[i]["href"]
            for i in range(len(find_all_list))
        }
        dict1.update(url_dict)
        dfs_list = pd.read_html(url)
        # サイト更新時にページのtableのデザインが変更していないかチェック
        if year >= 2022:
            try:
                if dfs_list[0].columns[0] != '企業名':
                    raise ValueError("銘柄一覧から取得したtableの値が「企業名」ではありません")
                elif dfs_list[1].columns[0] != '総合評価':
                    raise ValueError("銘柄一覧から取得したtableの値が「総合評価」ではありません")
            except ValueError as e:
                # エラーが発生した場合の処理
                print("Error: {}".format(e))
                lineno = inspect.currentframe().f_lineno
                print(f"エラーが発生した行番号: {lineno}")   
        time.sleep(np.random.randint(100,120)/100)
        for i in range(0, len(dfs_list) , 2):
            dfs_list_con = pd.concat([dfs_list[i], dfs_list[i+1]], axis=1)
            dfs_list_con["上場年"] = f"{year}"
            if year == 2019:
                dfs_list_con = dfs_list_con.rename(columns={'上場 市場': '上場市場'})
            if year >= 2020:
                dfs_list_con = dfs_list_con.rename(columns={'申し込み期間': '申し込み 期間', '初値上昇率': '初値 上昇率'})
            df_scraping_companies = pd.concat([df_scraping_companies, dfs_list_con],ignore_index=True)
            df_scraping_companies = df_scraping_companies.reset_index(drop=True)
        df_scraping_companies.drop(df_scraping_companies[df_scraping_companies['企業名'] == "企業名"].index , inplace=True)
        df_scraping_companies.drop(df_scraping_companies[df_scraping_companies['初値'] == "初値"].index , inplace=True)
    if year != this_year:
        time.sleep(np.random.randint(100,120)/100)
#codeカラム作成
df_scraping_companies_code = df_scraping_companies["企業名"].str.extract(r'([0-9]{4})').fillna(0).astype(int)
df_scraping_companies["code"] = df_scraping_companies["銘柄 コード"].fillna(df_scraping_companies_code[0])

2011 出力中
2012 出力中
2013 出力中
2014 出力中
2015 出力中
2016 出力中
2017 出力中
2018 出力中
2019 出力中
2020 出力中
2021 出力中
2022 出力中


In [ ]:
# dict作成
# 空白削除
del dict1[" "]
del dict1[""]
del dict1["\n"]
# dictをdf化
df_scraping_url = pd.DataFrame.from_dict(dict1, orient='index')
# カラム名変更
df_scraping_url = df_scraping_url.rename(columns={0:"url"})
# 間違えているurlを変更
df_scraping_url.url[df_scraping_url.url == "https://www.ipokiso.com/https://www.ipokiso.com/company/2013/zigexn.html"] = "https://www.ipokiso.com/company/2013/zigexn.html"
df_scraping_url = df_scraping_url.rename_axis('index').reset_index()
df_scraping_url = df_scraping_url.rename(columns={'index':'company_name'})

In [ ]:
df_scraping_url.dtypes

In [ ]:
# 2
# seleniumでのデータ取得用ループ　評価
# chromeのwebdriverのinstallしブラウザを起動する 
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)
def make_company_info(url):
    val_list = []
    driver.get(url)
    #seleniumの場合ここでsleepしないと正しく取得出来ない場合がある
    time.sleep(np.random.randint(100,120)/100)
    # HTMLを文字コードをUTF-8に変換してから取得します。
    html_res = driver.page_source.encode('utf-8')
#ここでシステムエラー処理する？
    # if html_res.status_code != 200:
    #     print("requests.getでのurlのアクセスができていません")
    #     lineno = inspect.currentframe().f_lineno
    #     print(f"エラーが発生しました。行番号: {lineno}")
    #     sys.exit()
    soup = bs4(html_res, 'html.parser')
    # 全企業codeの取得
    try:
        code = int(re.search(r"[0-9]{4}", soup.title.text).group())
    except AttributeError:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = int(re.search(r"[0-9]{4}", f).group())
    # 3171のtitleのcodeが3172と誤字のための修正
    if code == 3172:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = int(re.search(r"[0-9]{4}", f).group())
    val_list.append(code)
    # 成長性等の評価取得
    table = soup.find('table',class_="company01")
    val = table.find_all('td')
    # valからカラムの値に入れたい◎等の値のみ抽出
    try:
        for i in range(4):
            if ">？<" in str(val[i]):
                keyword = "？"
            elif "/sannkaku02.gif" in str(val[i]):
                keyword = "△"
            elif "/sannkaku.gif" in str(val[i]):
                keyword = "△"
            elif "/sankaku.gif" in str(val[i]):
                keyword = "△"
            elif "/maru02.gif" in str(val[i]):
                keyword = "〇"
            elif "/maru.gif" in str(val[i]):
                keyword = "〇"
            elif "/2maru.gif" in str(val[i]):
                keyword = "◎"
            elif "/s.gif" in str(val[i]):
                keyword = "S"
            elif "/a.gif" in str(val[i]):
                keyword = "A"
            elif "/b.gif" in str(val[i]):
                keyword = "B"
            elif "/c.gif" in str(val[i]):
                keyword = "C"
            elif "/d.gif" in str(val[i]):
                keyword = "D"
            val_list.append(keyword)
    except AttributeError:
        val_list.extend([np.nan,np.nan,np.nan,np.nan])
        print(f"成長性listのエラー{code}")
    print(code)
    # アンケート調査結果の取得
    print(len(soup.find_all("div",class_="vote-bar")))
    # 6行の場合
    if len(soup.find_all("div",class_="vote-bar")) == 6:
        vote_list = []
        number_of_votes_list = soup.find_all("div",class_="vote-bar")
        for value in number_of_votes_list:
            value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
            val_list.append(value)
    # 5行の場合
    elif len(soup.find_all("div",class_="vote-bar")) == 5:
        vote_list = []
        val_list.append(np.nan)
        number_of_votes_list = soup.find_all("div",class_="vote-bar")
        for value in number_of_votes_list:
            value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
            val_list.append(value)
    # 10行の場合 5行のアンケートがなぜか２つ表示されている銘柄
    elif len(soup.find_all("div",class_="vote-bar")) == 10:
        vote_list = []
        val_list.append(np.nan)
        number_of_votes_list = soup.find_all("div",class_="vote-bar")
        number_of_votes_list = number_of_votes_list[0:5]
        for value in number_of_votes_list:
            value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
            val_list.append(value)
    # アンケートがない場合
    elif len(soup.find_all("div",class_="vote-bar")) == 0:
        val_list.extend([np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
    # その他の行数エラー検知
    else:
        print("票の価格帯の数が分岐にありません")
        driver.quit()
        raise Exception("Error: アンケートの価格帯の数がifの分岐にありません")
    # 今まで取得した情報をリストに格納
    evaluation_list.append(val_list)
df_scraping_evaluation = df_scraping_url.copy()
url_list = df_scraping_url.url.values
evaluation_list = []
[make_company_info(url) for url in url_list[800:850]] #ここで検証のurl数変更！
print(evaluation_list)
df_company_01 = pd.DataFrame(evaluation_list,columns=[
    "code",
    "成長性",
    "割安性",
    "話題性",
    "総合評価",
    "初値予想_+200%以上",
    "初値予想_+100%以上+200%未満",
    "初値予想_+50%以上+100%未満",
    "初値予想_+20%以上+50%未満",
    "初値予想_+0%以上+20%未満",
    "初値予想_+0%未満",
])
df_scraping_evaluation[df_company_01.columns] = df_company_01
print(df_scraping_evaluation)
driver.quit()

In [ ]:
# 3
def make_company_info_2(url):
    dfs_table_list = pd.read_html(url)
    print(dfs_table_list[1].iloc[0,1])
    time.sleep(np.random.randint(100,120)/100)
    # サイト更新時にデザイン変更がないか検知する（現状ワシントンホテルのみ違うためスルー）
    # 回避urlリスト
    avoidance_url_list = [
        "https://www.ipokiso.com//company/2019/washingtonhotel.html",#[0]に１つ余分に優待情報のtableある
        "https://www.ipokiso.com//company/2020/gmo-fg.html",#table[6][7]が[7][8]にずれている
        "https://www.ipokiso.com//company/2021/frontier.html",#table[6]が無い
        "https://www.ipokiso.com//company/2021/geolocation.html",#table[6]が無い
        "https://www.ipokiso.com//company/2015/gmo-media.html",#table[6][7]が[7][8]にずれている
        ]
    if url not in avoidance_url_list:
        try:
            if dfs_table_list[1].iloc[0,0] != "会社名":
                raise ValueError("銘柄一覧から取得したtableの値が「会社名」ではありません")
            elif dfs_table_list[2].iloc[0,0] != '想定価格':
                raise ValueError("銘柄一覧から取得したtableの値が「想定価格」ではありません")               
            elif dfs_table_list[3].iloc[0,0] != '抽選申込期間':
                raise ValueError("銘柄一覧から取得したtableの値が「抽選申込期間」ではありません")
            elif not '公募' in dfs_table_list[4].iloc[0,0]:
                raise ValueError("銘柄一覧から取得したtableの値が「公募株数等」ではありません")
            elif dfs_table_list[5].iloc[0,1] != '証券会社名':
                raise ValueError("銘柄一覧から取得したtableの値が「証券会社名」ではありません")
        except ValueError as e:
            # エラーが発生した場合の処理
            print("Error: {}".format(e))
            lineno = inspect.currentframe().f_lineno
            print(f"エラーが発生した行番号: {lineno}")
            print(f"エラーが発生したURL:{url}")
            print(dfs_table_list)
            sys.exit()
        # 会社名にREITと投資法人という文字列が無いことを判定する
        if not ("REIT" in dfs_table_list[1].iloc[0,1]) and not ("投資法人" in dfs_table_list[1].iloc[0,1]):
            try:
                if dfs_table_list[6].columns[0] != '株主名':
                    raise ValueError("銘柄一覧から取得したtableの値が「株主名」ではありません")
                elif not ("（百万円）" in dfs_table_list[7].iloc[1,0]) and not ("（千米ドル）" in dfs_table_list[7].iloc[1,0]):
                    raise ValueError("銘柄一覧から取得したtableの値が「財務データ等」ではありません")
            except ValueError as e:
                # エラーが発生した場合の処理
                print("Error: {}".format(e))
                lineno = inspect.currentframe().f_lineno
                print(f"エラーが発生した行番号: {lineno}")
                print(f"エラーが発生したURL:{url}")
                print(dfs_table_list)
                sys.exit()

    # ここからdfsの結合、整形コード
    df_con_list = []
    for i in range(len(dfs_table_list)):
        # 基本情報
        if i == 1:
            # tableの位置指定
            df_table = dfs_table_list[i]
            # table位置が違うものの例外処理
            if df_table.iloc[0,0] != "会社名":
                df_table = dfs_table_list[i + 1]
                print(url)
            # df整形：基本情報
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            # 追加する1件分の基本情報のdf作成
            df_table_1 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_1)
            # 表記の揺れ統一のための修正

        # IPO日程と価格決定（初値予想）
        if i == 2:
            # tableの位置指定
            df_table = dfs_table_list[i]
            # table位置が違うものの例外処理
            if dfs_table_list[2].iloc[0,0] != '想定価格':
                df_table = dfs_table_list[i + 1]
                print(url)
            # df整形：IPO日程と価格決定（初値予想）
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_2 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_2)
            # 表記の揺れ統一のための修正
            
        # IPOスケジュール
        if i == 3:
            # tableの位置指定 # tableの位置指定
            df_table = dfs_table_list[i]
             # table位置が違うものの例外処理
            if dfs_table_list[3].iloc[0,0] != '抽選申込期間':
                df_table = dfs_table_list[i + 1]
                print(url)
            
            # df整形：IPOスケジュール
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_3 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_3)
            # 表記の揺れ統一のための修正
        # IPO当選株数
        if i == 4:
            # tableの位置指定
            df_table = dfs_table_list[i]
             # table位置が違うものの例外処理
            if not '公募' in dfs_table_list[4].iloc[0,0]:
                df_table = dfs_table_list[i + 1]
                print(url)
            # ワシントンホテルのみ[1]に優待情報があるためtableをずらしている
            if url == "https://www.ipokiso.com//company/2019/washingtonhotel.html":
                df_table = dfs_table_list[i + 1]
            
            # df整形：IPO当選株数
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_4 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_4)
            # 表記の揺れ統一のための修正

        # 幹事証券リスト（管理人独自予想あり）
        if i == 5:
            # tableの位置指定
            df_table = dfs_table_list[i]
             # table位置が違うものの例外処理
            if dfs_table_list[5].iloc[0,1] != '証券会社名':
                df_table = dfs_table_list[i + 1]
                print(url)
            # df整形：幹事証券リスト（管理人独自予想あり）
            df_table_5 = pd.DataFrame()
            for j in range(1,len(df_table.iloc[:,0])):
                val = df_table.iloc[j,:].values
                cols = df_table.iloc[0,:]
                cols[0] = "幹事種類"
                cols = [f"{col}_{j}" for col in cols]
                df_add = pd.DataFrame([val],columns=cols)
                df_table_5 = pd.concat([df_table_5, df_add], axis=1)
            df_con_list.append(df_table_5)

        # 株主構成、ロックアップなど
        if i == 6:
            if dfs_table_list[1].iloc[0,0] == "会社名":
                company_name = dfs_table_list[1].iloc[0,1]
            elif dfs_table_list[2].iloc[0,0] == "会社名":
                company_name = dfs_table_list[2].iloc[0,1]
            if not ("REIT" in company_name or "投資法人" in company_name or  ("福証" in company_name and not "東証" in company_name)):
                # tableの位置指定
                df_table = dfs_table_list[i]
                # table位置が違うものの例外処理
                if dfs_table_list[6].columns[0] != '株主名':
                    df_table = dfs_table_list[i + 1]
                    print(url)
                df_table_6 = pd.DataFrame()
                for k in range(0,len(df_table.iloc[:,0])):
                    val = df_table.iloc[k,:].values
                    cols = df_table.columns
                    cols = [f"{col}_{k+1}" for col in cols]
                    df = pd.DataFrame([val],columns=cols)
                    df_table_6 = pd.concat([df_table_6, df], axis=1)
                df_con_list.append(df_table_6)

            # 企業業績のデータ（5年分）
        if i == 7:
            if not ("REIT" in company_name or "投資法人" in company_name):

                # tableの位置指定
                df_table = dfs_table_list[i]
                # table位置が違うものの例外処理
                if not ("（百万円）" in dfs_table_list[7].iloc[1,0]) and not ("（千米ドル）" in dfs_table_list[7].iloc[1,0]):
                    df_table = dfs_table_list[i + 1]
                    print(url)
                
                df_table_7 = pd.DataFrame()
                for h in range(1,len(df_table.T.iloc[:,0])):
                    val = df_table.T.iloc[h,:].values
                    # カラム名の誤字修正
                    cols = df_table.T.iloc[0,:]
                    cols[0] = "年月"
                    cols = [f"{col}_{h}年目" for col in cols]
                    df = pd.DataFrame([val],columns=cols)
                    df_table_7 = pd.concat([df_table_7, df], axis=1)
                df_con_list.append(df_table_7)


            # 全件のdfに追加していく
    df_table_all = pd.concat(df_con_list,axis=1)
    print(url)
    print("finish")
    return df_table_all
df_scraping_table_all = pd.DataFrame()
url_list = df_scraping_url.url.values
for url in url_list[0:10]:#ここでurl数変更
    df_table_all = make_company_info_2(url) 
    df_scraping_table_all = pd.concat([df_scraping_table_all,df_table_all],axis=0,ignore_index=True)
print(df_scraping_table_all)

In [ ]:
# 時短のためcode含めたcsv読み込み　コード完成後に削除する
# df_scraping_evaluation = pd.read_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_scraping_url.csv", index_col=0)
# df_scraping_evaluation

12/15 dfの作成　4つ

In [ ]:
df_scraping_companies.rename(columns={'銘柄 コード': 'code'}, inplace=True)
df_scraping_companies["code"] = df_scraping_companies["code"].astype(float)
df_scraping_companies.dtypes

In [ ]:
df_scraping_url

In [ ]:
df_scraping_evaluation

In [ ]:
df_scraping_table_all

In [ ]:
df_scraping_companies["code"]

In [ ]:
df_scraping_eva_tab = pd.concat([df_scraping_evaluation,df_scraping_table_all],axis=1)

In [ ]:
df_scraping_eva_tab

In [ ]:
df_scraping_companies["code"].values[300:600]

In [ ]:
df_scraping_eva_tab["code"].values[0:100]

次はここから！2022/12/19予定
１、codeの値の型がintやstrで不統一、取得時にintで統一
２，df_scraping_companies["code"]は最初からコードと会社名を加工して新カラムcodeをつくる
３，df_scraping_eva_tab["code"]はスクレイピング時にint型を指定するように変更する。
４，mergeでscraping_allで年がとれるように達成する。
５，全てのscrapingの情報から4つのdfの分割等を実施する。

In [ ]:
df_scraping_all = pd.merge(df_scraping_eva_tab, df_scraping_companies, on="code", how="inner")
df_scraping_all

In [ ]:
# 評価+ 会社設立年 + 市場(G,P,S,…)　+ IPO日程と価格決定(初値予測) + 初値予想アンケート
df_assessments = pd.concat([df_scraping_evaluation.loc[:, [
    "code",
    "成長性",
    "割安性",
    "話題性",
    "総合評価",
    "初値予想_+200%以上",
    "初値予想_+100%以上+200%未満",
    "初値予想_+50%以上+100%未満",
    "初値予想_+20%以上+50%未満",
    "初値予想_+0%以上+20%未満",
    "初値予想_+0%未満"
]],
df_scraping_table_all.loc[:, [
    "会社設立",
    "会社名",#市場に変形する
    "想定価格",
    "仮条件",
    "公募価格",
    "初値予想（独自）",
    "初値"
]]],axis=1)
df_assessments

In [ ]:
# 重複チェック
df_assessments['code'].value_counts(ascending=False)


In [ ]:
df_assessments['code'].count()

In [ ]:
df_scraping_all.to_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_scraping_all.csv", encoding='utf-8_sig',index=False)

In [ ]:
# IPOスケジュール + IPO当選株数 
df_schedule = pd.concat([df_scraping_evaluation.loc[:, [
    "code",
]],
df_scraping_table_all.loc[:, [
    "抽選申込期間",
    "当選発表日",
    "購入申込期間",
    "上場日",
    "公募株数",
    "売出株数（OA含む）",
    "当選株数合計"
]]],axis=1)
df_schedule

In [ ]:
df_schedule.dtypes

In [ ]:
df_schedule[df_schedule["code"] == "3170"]

In [ ]:
# 幹事証券リスト（管理人独自予想あり）
df_underwriters = pd.concat([df_scraping_evaluation.loc[:, [
    "code",
]],
df_scraping_table_all.filter(regex="幹事種類_|証券会社名_|割当率_|割当株数_|当選本数 （枚）_|完全抽選本数 （予想）_")
],axis=1)
df_underwriters

In [ ]:
df_underwriters.dtypes

In [ ]:
# 株主構成 + ロックアップ[6]
df_shareholders = pd.concat([df_scraping_evaluation.loc[:, [
    "code",
]],
df_scraping_table_all.filter(regex="株主名_|比率_|ロック アップ_|ロック  アップ_")
],axis=1)
df_shareholders

ここ以下で作成　12/13～

In [ ]:
#初値予想アンケートの取得実験用　12/13～　これをもとに追加、その後削除する
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)
url =  rf"https://www.ipokiso.com/company/2022/beex.html"
driver.get(url)
html_res = driver.page_source.encode('utf-8')
soup = bs4(html_res, 'html.parser')

In [ ]:
# アンケート価格リスト
soup.find_all("p",class_="vote-left")

In [ ]:
# アンケートの票リスト
soup.find_all("div",class_="vote-bar")

In [ ]:
len(soup.find_all("div",class_="vote-bar"))

In [ ]:
vote_values_list = soup.find_all("p",class_="vote-left")
number_of_votes_list = soup.find_all("div",class_="vote-bar")

In [ ]:
votes_list = []
for value in number_of_votes_list:
    # print(value)
    value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
    votes_list.append(value)

In [ ]:
# driver.quit()

In [ ]:
df_scraping_table_all.to_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_scraping_table_all.csv", encoding='utf-8_sig',index=False)

df_scraping_table_all

これを元に作成2022/12/09~ ↓　全銘柄チェック、その後、例外の銘柄のtableの変更実施


以上本番用

In [ ]:
""" 
# csv変換 時短のためこれを読み込む
df_code = pd.DataFrame(code_list,columns=["code"])
df_scraping_url[df_code.columns] = df_code.values
df_scraping_evaluation.to_csv("df_scraping_evaluation", encoding='utf-8_sig')
"""

In [ ]:
# 時短のためcode含めたcsv読み込み
df_scraping_evaluation = pd.read_csv("df_scraping_url.csv", index_col=0)
df_scraping_evaluation

- IPO当選株数のデータ取得

backup

In [ ]:
# backup用
# requests.getでのデータ取得用ループ　評価
def make_company_info(url):
    html_res = requests.get(url)
    if html_res.status_code != 200:
        print("requests.getでのurlのアクセスができていません")
        lineno = inspect.currentframe().f_lineno
        print(f"エラーが発生しました。行番号: {lineno}")
        sys.exit()
    time.sleep(np.random.randint(100,120)/100)
    soup = bs4(html_res.content, 'html.parser')
    # 全企業codeの取得
    val_list = []
    try:
        code = re.search(r"[0-9]{4}", soup.title.text).group()
    except AttributeError:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = re.search(r"[0-9]{4}", f).group()
    val_list.append(code)
    # 成長性等の評価取得
    table = soup.find('table',class_="company01")
    val = table.find_all('td')
    # valからカラムの値に入れたい◎等の値のみ抽出
    try:
        for i in range(4):
            if ">？<" in str(val[i]):
                keyword = "？"
            elif "/sannkaku02.gif" in str(val[i]):
                keyword = "△"
            elif "/sannkaku.gif" in str(val[i]):
                keyword = "△"
            elif "/sankaku.gif" in str(val[i]):
                keyword = "△"
            elif "/maru02.gif" in str(val[i]):
                keyword = "〇"
            elif "/maru.gif" in str(val[i]):
                keyword = "〇"
            elif "/2maru.gif" in str(val[i]):
                keyword = "◎"
            elif "/s.gif" in str(val[i]):
                keyword = "S"
            elif "/a.gif" in str(val[i]):
                keyword = "A"
            elif "/b.gif" in str(val[i]):
                keyword = "B"
            elif "/c.gif" in str(val[i]):
                keyword = "C"
            elif "/d.gif" in str(val[i]):
                keyword = "D"
            val_list.append(keyword)
    except AttributeError:
        val_list = [np.nan,np.nan,np.nan,np.nan]
        print(f"成長性listのエラー{code}")
    evaluation_list.append(val_list)
    print(code)
df_dict
url_list = df_dict.url.values
evaluation_list = []
[make_company_info(url) for url in url_list[0:3]] #ここで検証のurl数変更！
df_company_01 = pd.DataFrame(evaluation_list,columns=["code","成長性","割安性","話題性","総合評価"])
df_dict[df_company_01.columns] = df_company_01
print(df_dict)